# 获取指定城市地铁路线

In [8]:
# 获取指定城市的地铁路线
import requests
from bs4 import BeautifulSoup

# 根据request_url得到soup
def get_page_content(request_url):
    # 得到页面的内容
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    html = requests.get(request_url, headers=headers, timeout=10)
    content = html.text
    #print(content)
    # 通过content创建BS对象
    soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')
    return soup
request_url = 'https://ditie.mapbar.com/beijing_line/'
soup = get_page_content(request_url)
soup

/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:203: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")



<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>北京地铁线路大全_北京地铁线路图_北京地铁 最新-图吧地铁</title>
<meta content="北京地铁线路图,北京地铁线路,北京地铁,北京地铁线路图 最新" name="keywords"/>
<meta content="【北京地铁线路大全】北京图吧地铁，提供最新北京所有地铁线路查询，真实北京地铁线路图等信息。" name="description"/>
<link href="//r.mapbar.com/ditie/css/style.css" rel="stylesheet" type="text/css"/>
<link href="//r.mapbar.com/ditie/css/base.css" rel="stylesheet" type="text/css"/>
<link href="//r.mapbar.com/ditie/css/style_time.css" rel="stylesheet" type="text/css"/>
<script src="//api.mapbar.com/api/mapbar31.3.js" type="text/javascript"></script>
<script src="//r.mapbar.com/ditie/js/jquery-1.9.1.min.js" type="text/javascript"></script>
<script src="//cbjs.baidu.com/js/m.js" type="text/javascript"></script>
<script type="text/javascript">
		BAIDU_CLB_preloadSlots("934189","934191","946594");
	</script>
<script src="//s.union.360.cn/100009.js" type="text/javascript"></script>
</head>
<body>
<input id="cncity" type="hidden" value="北京市"/>
<div class

In [9]:
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=['name','site'])
subways = soup.find_all('div', class_='station')
for subway in subways:
    route_name = subway.find('strong', class_='bolder').text
    routes = subway.find('ul')
    routes = routes.find_all('a')
    for route in routes:
        #print(route.text)
        temp = {'name': route.text, 'site': route_name}
        df = df.append(temp, ignore_index=True)
df       
    

,name,site
0,四惠站,地铁八通线
1,四惠东站,地铁八通线
2,高碑店站,地铁八通线
3,传媒大学站,地铁八通线
4,双桥站,地铁八通线
...,...,...
346,永丰站,地铁16号线
347,永丰南站,地铁16号线
348,西北旺站,地铁16号线
349,马连洼站,地铁16号线


In [10]:
df['city'] = '北京'
df.to_csv('./subway.csv', index=False)

# 添加经度longitude、纬度latitude

In [11]:
# 添加经度longitude、纬度latitude 
import re
key = 'd31ca20f3fc9f31d76e20359404a0783'

def get_location(keyword, city):
    request_url = 'http://restapi.amap.com/v3/place/text?key=' + key + \
        '&keywords=' + keyword + '&types=&city=' + city + '&children=1&offset=1&page=1&extensions=all'
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    data = requests.get(request_url, headers=headers, timeout=10)
    data.encoding='utf-8'
    data = data.text
    #print(data)
    """
        后面多一个？表示懒惰模式
        .*具有贪婪的性质，首先匹配到不能匹配为止
        .*?则相反，一个匹配以后，就往下执行后续的正则
    """
    pattern = 'location":"(.*?),(.*?)"'
    # 得到经纬度
    result = re.findall(pattern, data)
    # 如果有多个，取第一位置
    try:
        return result[0][0], result[0][1]
    except:
        return get_location(keyword.replace('站', ''),city)

In [12]:
get_location('石门站', '北京')

('116.640923', '40.129319')

In [13]:
#df['longitude'], df['latitude'] = None, None
for index, row in df.iterrows():
    longitude, latitude = get_location(row['name'], row['city'])
    df.loc[index, 'longitude'] = float(longitude)
    df.loc[index, 'latitude'] = float(latitude)
    print(longitude, latitude)

116.498489 39.908279
116.514954 39.908051
115.858901 39.328727
116.554931 39.909548
116.569498 39.901066
116.598786 39.908696
116.617844 39.906559
116.636748 39.904070
116.646183 39.893199
116.662943 39.888520
116.669083 39.883915
116.679099 39.875796
116.687215 39.871679
116.306169 40.052307
116.294575 40.094705
116.313248 40.104775
116.293671 40.130836
116.288410 40.148186
116.279991 40.164629
116.287159 40.207766
116.282712 40.221261
116.262840 40.221475
116.194657 40.190277
116.207894 40.239493
116.194237 40.244382
116.125346 39.723503
116.140429 39.723478
116.155903 39.723547
116.176119 39.722872
116.183056 39.729827
116.180550 39.746964
116.189793 39.760327
116.212569 39.763968
116.218561 39.794460
116.291802 39.807984
116.302856 39.814140
116.428645 39.843724
116.447857 39.834035
116.459255 39.828751
116.460624 39.807513
116.481035 39.803228
116.491242 39.807005
116.505292 39.802860
116.513118 39.793611
116.522169 39.782348
116.539213 39.772623
116.562750 39.783930
116.581150 39

In [15]:
df.to_csv('./subway.csv',index=False)